In [1]:
import os
os.chdir('../..')
os.getcwd()

'/home/user/code'

In [2]:
import pandas as pd
import numpy as np
from textprocessor.normalization import *
from textprocessor.normalization import _NormalizeMeta
from functools import partial

from textprocessor.text_transform import *
from textprocessor.statistic import *

# Split Dataset

In [3]:
BASE_DATASET_DIR = "dataset/qaset"
NORM_DATASET_DIR = "dataset/qaset/normset/set1"

Tiền xử lí dataset
* Clean text: loại bỏ các tag <> (VD: `< nowiki > * < / nowiki >, <br>, <ref> </ref>, ...`)

Chia file train.json thành 3 file train_set.csv, valid_set.csv, test_set.csv (0.7:0.15:0.15) với tỉ lệ label `True, False` giống nhau. Xem file `text_processor\text_transform`

In [4]:
train_df = load_train_data(BASE_DATASET_DIR)

In [5]:
names = ['train', 'valid', 'test']
sz = [0.7, 0.15, 0.15]
dfs = split_ds(train_df, sz)

In [6]:
dfs[1].iloc[0]['text']

'Ba Lan ( tiếng Ba Lan : " Polska " ) , tên chính thức là Cộng hoà Ba Lan ( tiếng Ba Lan : " Rzeczpospolita Polska " ) , là một quốc gia ở Trung Âu , tiếp giáp với các nước Đức , Slovakia , Cộng hoà Séc , Ukraina , Belarus , Litva , Nga và biển Baltic ; diện tích 312.685  km2 , dân số 38,56 triệu gần như thuần chủng , đa phần ( 95% ) theo đạo Công giáo Rôma được truyền bá vào đây khi nhà nước Ba Lan đầu tiên hình thành vào thế kỷ thứ X. Lãnh thổ Ba Lan trong lịch sử có nhiều thay đổi , biên giới hiện nay có từ sau Chiến tranh thế giới thứ hai theo Thoả thuận Yalta .'

In [7]:
cols = {'question':'q', 'text':'t'}
for i, na in enumerate(names):
    dfs[i].to_csv(f'{NORM_DATASET_DIR}/{na}_set.csv')
    t = create_feature_df(dfs[i], drop=True, cols=cols)
    t.to_csv(f'{NORM_DATASET_DIR}/{na}_feature.csv')

100%|██████████| 2715/2715 [00:37<00:00, 71.52it/s]


# Text Processing

## Filter punctuation and stopwords

In [8]:
train_feature_df = pd.read_csv(f'{NORM_DATASET_DIR}/train_feature.csv')
valid_feature_df = pd.read_csv(f'{NORM_DATASET_DIR}/valid_feature.csv')
test_feature_df = pd.read_csv(f'{NORM_DATASET_DIR}/test_feature.csv')
# Them dataset gg dich tu squad 2.0
# aug_df = pd.read_csv('dataset/qaset/train_feature_squad.csv').sample(frac=1).reset_index(drop=True)
stopwords = pd.read_csv('textprocessor/stopwords.csv')
train_feature_df.head(5)

,Unnamed: 0,id,title,label,q_toks,q_pos,q_ner,t_toks,t_pos,t_ner
0,0,u8-1556876599_0,Tam Cốc - Bích Động,True,Nam thiên đệ nhất động là tên gọi của danh_lam...,Np V N A N V N V E N P,B-PER O O O O O O O O O O,"Bích_Động nằm cách bến Tam_Cốc 2 km , có nghĩa...",Np V V N Np M Nu CH V N C CH N A CH CH V N E N...,B-LOC O O O O O O O O O O O O O O O O O O O B-...
1,1,u3-1559034795_0,Hệ động vật Việt Nam,True,Thanh_Hoá có vườn_quốc_gia nào,N V N P,O O O O,Một kết_quả quan_trọng là đã xác_định các khu_...,M N A V R V L N N C N N V V N L Np E Np CH N M...,O O O O O O O O O O O O O O O O O O B-LOC O O ...
2,2,u1-1556061605_3,Everest,True,Đỉnh núi cao nhất thế_giới hiện_nay tên là gì,N N A A N N N V P,O O O O O O O O O,Đỉnh Everest ( tên khác : đỉnh Chomolungma ) l...,N Np CH N A CH N Np CH V N N A R E N V E N N C...,O B-LOC O O O O O O O O O O O O O B-LOC O O O ...
3,3,u1-1554938643_0,Tinh bột,True,Công_thức_hoá_học của tinh_bột là gì,N E N V P,O O O O O,Tinh_bột tiếng Hy_Lạp là amidon ( CAS # 9005-2...,N N Np V N CH Ny CH M CH N CH CH Ny CH CH V M ...,O B-MISC I-MISC O O O O O O O O O O O O O O O ...
4,4,u3-1558929435_0,NaN,True,Trạng_Bùng_Phùng_Khắc Khoan từng làm quan cho ...,N Np R V N E N N P,O B-ORG O O O O O O O,"Phùng_Khắc_Khoan : Công_việc Quan nhà Hậu_Lê ,...",N CH N Np N Np CH Np,O O O O B-MISC I-MISC O O


In [9]:
group={'q_toks': ['q_pos', 'q_ner'], 't_toks': ['t_pos', 't_ner']}
# Loại bỏ punctuation, không bỏ stopword do cần context meaning
filed = filter_toknfeat_df(train_feature_df, [is_punctuation], group=group,reverse_check=True)
filed.to_csv(f'{NORM_DATASET_DIR}/train_filted_feature.csv')
filed = filter_toknfeat_df(valid_feature_df, [is_punctuation], group=group,reverse_check=True)
filed.to_csv(f'{NORM_DATASET_DIR}/valid_filted_feature.csv')
filed = filter_toknfeat_df(test_feature_df, [is_punctuation], group=group,reverse_check=True)
filed.to_csv(f'{NORM_DATASET_DIR}/test_filted_feature.csv')

100%|██████████| 2715/2715 [00:11<00:00, 230.99it/s]


In [3]:
# group={'q_toks': ['q_pos', 'q_ner'], 't_toks': ['t_pos', 't_ner']}
# filed = filter_toknfeat_df(aug_df, [is_punctuation], group=group,reverse_check=True)
# filed.to_csv('dataset/qaset/aug_feature_set2.csv')

# Question corpus

In [3]:
train_feature_df = pd.read_csv(f'{NORM_DATASET_DIR}/train_filted_feature.csv')
train_feature_df.head(2)

In [5]:
df2corpus('dataset/corpus/zl_train_quest.txt',train_feature_df, cols=['q_toks'])

In [50]:
cols = {'question':'q'}
group={'q_toks': ['q_pos', 'q_ner']}
test_df = load_test_data('dataset/qaset')
test_df = test_df.drop(columns=['text', 'title'])
test_df = test_df.groupby(['question'])
test_df = test_df.count()
test_df = pd.DataFrame(test_df['id'].keys())
t = create_feature_df(test_df, drop=True, cols=cols)
filed = filter_toknfeat_df(t, [is_punctuation], group=group,reverse_check=True)
# t.to_csv(f'dataset/qaset/{na}_feature_set2.csv')
df2corpus('dataset/corpus/zl_test_quest.txt',t, cols=['q_toks'])

# Load các dataset

In [3]:
NORM_DATASET_DIR = "dataset/qaset/normset/set1"

In [4]:
orig_set = ZaloTextPreprocess.load_train_data(DATASET_DIR)
train_set = pd.read_csv(DATASET_DIR + '/train_set.csv')
valid_set = pd.read_csv(DATASET_DIR + '/valid_set.csv')
test_set  = pd.read_csv(DATASET_DIR + '/test_set.csv')

In [5]:
# 706 title và 2 text rỗng 
print('-------Train---------')
print(train_set.isnull().sum())
print('\n-------Valid---------')
print(valid_set.isnull().sum())
# print('\n-------Test---------')
# print(test_set.isnull().sum())

In [6]:
print('2 câu không có text là')
tmp = train_set.replace(np.nan, '', regex=True)
tmp[tmp['text']=='']

In [15]:
aug_df = pd.read_csv('dataset/qaset/train_feature_squad.csv').sample(frac=1).reset_index(drop=True)
tmp = aug_df.replace(np.nan, '', regex=True)
tmp[(tmp['t_toks'] != '') & (tmp['q_toks'] != '')]

In [11]:
tmp.applymap(type).eq(str).all()

# Feature

In [7]:
# train_feature = pd.read_csv('train_feature.csv')

PoS và NER

In [12]:
# zl2corpus('train_corpus.txt', train_feature, cols=['q_toks', 't_toks'], pros=[remove_punctuation, lambda x: x.lower()])

In [9]:
# def lower(text):
#     toks = text.split()
#     out = ''
#     for tok in toks:
#         tl = tok.lower()
#         tlcf ='_'.join([t.capitalize() for t in tl.split('_')])
#         if tl != tok:
#             if tl.capitalize() == tok:
#                 out += '<cap> '
#             elif tlcf == tok:
#                 out += '<capw> '
#             else:
#                 out += '<up> '
#         out += tok.lower() + ' '
#     return out.strip()



In [11]:
# c = token_statistic('train_corpus.txt', 'train_corpus.stat')